# SuAVE Image to Label SVM Prediction Model

In [31]:
%%javascript
function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("survey_url='".concat(getQueryStringValue("surveyurl")).concat("'"));
IPython.notebook.kernel.execute("user='".concat(getQueryStringValue("user")).concat("'"));
//IPython.notebook.kernel.execute("csv_file='".concat(getQueryStringValue("csv")).concat("'")); 
IPython.notebook.kernel.execute("dzc_file='".concat(getQueryStringValue("dzc")).concat("'")); 
IPython.notebook.kernel.execute("params='".concat(getQueryStringValue("params")).concat("'")); 
IPython.notebook.kernel.execute("active_object='".concat(getQueryStringValue("activeobject")).concat("'")); 

<IPython.core.display.Javascript object>

### Import all packages

In [32]:
import warnings
warnings.filterwarnings('ignore')  # "error", "ignore", "always", "default", "module" or "once"

# More imports
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import csv
import pandas as pd
import re
import cv2
import os

# import the necessary packages for SVM predictor
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import imutils
from sklearn.externals import joblib

# import histogram func
from histogram import Histograms

# Import widget functionality
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [33]:
# Testing Cells
csv_file = 'balthus.csv'

## Run and choose the column name that coresponds with the column label to be predicted

In [20]:
# use csv file to grab images/labels
df = pd.read_csv(csv_file)

# Choose column of label for prediction
toPredict = list(df.columns.values)

pred_menu = {}
for i in range(0, len(toPredict)):
    pred_menu[toPredict[i]] = toPredict[i]

def f(predictions_menu):
    return predictions_menu

out2 = interact(f, predictions_menu=pred_menu)

interactive(children=(Dropdown(description='predictions_menu', options={'#href': '#href', 'Dimensions': 'Dimen…

## Choose type of histogram to extract from images

In [21]:
hist_type = { 
    'HSV Color Histogram': 0,
    'Blue Histogram': 1,
    'Red Histogram': 2,
    'Green Histogram': 3
    }

def f(histogram_type):
    return histogram_type
typeH= interact(f, histogram_type=hist_type);

interactive(children=(Dropdown(description='histogram_type', options={'Blue Histogram': 1, 'Red Histogram': 2,…

## Get histogram from each image in the dataset

In [22]:
# init the image suffix
suffix = '.jpg'
labels = []

labelHeader = out2.widget.result

# grab chosen column names
nameCol = df['#img']
predCol = df[labelHeader]

# add all fabric columns to the y set
for i in range (0,len(predCol)):
    labels.append(predCol[i])
# create new hist_list
hist_list = []
# gather images from path created from file names in csv file
typeH = typeH.widget.result
for i in range (0,len(nameCol)):
    base_filename = nameCol[i]
    fileName = os.path.join("./Balthus/", base_filename + suffix)
    im = cv2.imread(fileName)  
    if typeH == 0:
        hist = Histograms.extract_color_histogram(im)
    elif typeH == 1: 
        hist = Histograms.extract_blue_histogram(im)
    elif typeH == 2: 
        hist = Histograms.extract_red_histogram(im)
    elif typeH == 3: 
        hist = Histograms.extract_green_histogram(im)
    hist_list.append(hist)
    
# transform labels into numerical system
le = LabelEncoder()
labels = le.fit_transform(predCol)

# separate data into test/train sets for features/labels
(xtrain, xtest, ytrain, ytest) = train_test_split(np.array(hist_list),labels, test_size = 0.5)

# Train the linear regression classifier
model2 = LinearSVC()
model2.fit(xtrain, ytrain)

# Calculate predictions on the data set
predictions = model2.predict(np.array(hist_list))
print(classification_report(labels, predictions, target_names = le.classes_))

                    precision    recall  f1-score   support

         cityscape       0.00      0.00      0.00         1
    genre painting       0.60      0.84      0.70        38
         landscape       0.82      0.64      0.72        14
 literary painting       0.50      0.50      0.50         2
nude painting (nu)       0.56      0.50      0.53        18
          portrait       0.83      0.71      0.77         7
     self-portrait       1.00      0.50      0.67         2
  sketch and study       1.00      0.38      0.55         8
        still life       0.50      0.50      0.50         4
 symbolic painting       0.00      0.00      0.00         2

       avg / total       0.66      0.65      0.63        96



In [29]:
# save the label, 
fileName = labelHeader + '_' + csv_file[:-4] + '_svmModel' + '.txt'
file = open(fileName,"w") 
file.write(str(typeH))
file.write(out2.widget.result)
file.close()
joblib.dump(model2, 'svmModel.pkl') 

['svmModel.pkl']

In [13]:
typeH

0

In [24]:
labelHeader

'Genre'

In [25]:
labelHeader + '_' + csv_file[:-4] + 'svmModel' + '.txt'

'Genre_balthussvmModel.txt'